In [ ]:
import os
import json
import turicreate as tc
import pandas as pd
import coremltools
import numpy as np
from PIL import Image

In [ ]:
# Prepare data creation pipeline
base_bkgs_path = "Dashboards/"
icons_path = "Icons/"
icon_images = [f for f in os.listdir(icons_path) if not f.startswith(".")]
bkg_images = [f for f in os.listdir(base_bkgs_path) if not f.startswith(".")]
sizes = [0.4, 0.6, 0.8, 1, 1.2] # different icon sizes to use

# Location where training data will get dumped
output_images = "NewTrainingImages/"

# Check initial size of all the icons to start
for d in icon_images:
    d_path = icons_path + d
    i = Image.open(d_path)
    print(d_path, i.size)

In [ ]:
def get_icon_positions(icon, bkg, count=1):
    icon_w, icon_h = [], []
    x_positions, y_positions = [], []
    bkg_w, bkg_h = bkg.size
    # Rescale our icon to have a couple different sizes
    icon_sizes = [tuple([int(s*x) for x in icon.size]) for s in sizes]
    for w, h in icon_sizes:
        icon_w.extend([w]*count)
        icon_h.extend([h]*count)
        max_x, max_y = bkg_w-w, bkg_h-h
        x_positions.extend(list(np.random.randint(0, max_x, count)))
        y_positions.extend(list(np.random.randint(0, max_y, count)))
    return icon_h, icon_w, x_positions, y_positions


def get_box(icon_w, icon_h, max_x, max_y):
    x1, y1 = np.random.randint(0, max_x, 1), np.random.randint(0, max_y, 1)
    x2, y2 = x1 + icon_w, y1 + icon_h
    return [x1[0], y1[0], x2[0], y2[0]]


# check if two boxes intersect
def intersects(box, new_box):
    box_x1, box_y1, box_x2, box_y2 = box
    x1, y1, x2, y2 = new_box
    return not (box_x2 < x1 or box_x1 > x2 or box_y1 > y2 or box_y2 < y1)


def get_group_icon_positions(icon_group, bkg):
    bkg_w, bkg_h = bkg.size
    boxes = []
    icons = [Image.open(icons_path + icon_images[i]) for i in icon_group]
    icon_sizes = [tuple([int(0.6*x) for x in i.size]) for i in icons]
    for w, h in icon_sizes:
        # set background image boundaries
        max_x, max_y = bkg_w-w, bkg_h-h
        # get new box coordinates for the icon on the bkg
        while True:
            new_box = get_box(w, h, max_x, max_y)
            for box in boxes:
                res = intersects(box, new_box)
                if res:
                    break
                    
            else:
                break  # only executed if the inner loop did NOT break
            print("retrying a new icon box")
            continue  # only executed if the inner loop DID break
        # append our new box 
        boxes.append(new_box)
    return icon_sizes, boxes

In [ ]:
# Prepare for data augmentation
count_per_size = 4
annotations = []
n = 1

# Make synthetic training data
for bkg in bkg_images:
    if bkg.startswith("."):
        continue
    
    # Load the background image
    bkg_path = base_bkgs_path + bkg
    bkg_img = Image.open(bkg_path)
    bkg_x, bkg_y = bkg_img.size

    # Do single icons first
    for i in icon_images:
        # Load the single icon
        i_path = icons_path + i
        icon_img = Image.open(i_path)
        
        # Get an array of random icon positions (top-left corner)
        icon_h, icon_w, x_pos, y_pos = get_icon_positions(icon=icon_img, bkg=bkg_img, count=count_per_size)
        
        # Create synthetic images based on positions
        for h, w, x, y in zip(icon_h, icon_w, x_pos, y_pos):
            # Make annotation
            ann = [{'coordinates': {'height': h, 'width': w, 'x': x+(0.5*w), 'y': y+(0.5*h)}, 'label': i.split(".png")[0]}]
            # Copy background
            bkg_w_icon = bkg_img.copy()
            # Adjust icon size
            new_icon = icon_img.resize(size=(w, h))
            # Paste on the icon
            bkg_w_icon.paste(new_icon, (x, y))
            output_fp = output_images + str(n) + ".png"
            # Save the image
            bkg_w_icon.save(fp=output_fp, format="png")
            # Save the annotation data
            annotations.append({
                "path": output_fp, 
                "annotations": ann
            })
            print(n)
            n += 1
            
 
    # 24 Groupings of 2-4 icons together on a single background
    groups = [np.random.randint(0, len(icon_images) -1, np.random.randint(2, 5, 1)) for r in range(2*len(icon_images))]
    # For each group of icons
    for group in groups:
        # Get sizes and positions
        icon_sizes, boxes = get_group_icon_positions(group, bkg_img)
        ann = []
        bkg_w_icon = bkg_img.copy()
        
        # For each icon in the group
        for i, size, box in zip(group, icon_sizes, boxes):
            # Get the icon
            icon = Image.open(icons_path + icon_images[i])
            icon_w, icon_h = size
            # Resize it as needed
            new_icon = icon.resize((icon_w, icon_h))
            x_pos, y_pos = box[:2]
            
            # Add icon annotations
            annot = {
                    'coordinates': {
                        'height': icon_h,
                        'width': icon_w,
                        'x': int(x_pos+(0.5*icon_w)),
                        'y': int(y_pos+(0.5*icon_h))
                    },
                    'label': icon_images[i].split(".png")[0]
                }
            ann.append(annot)
            # Paste the icon to the background
            bkg_w_icon.paste(new_icon, (x_pos, y_pos))

        output_fp = output_images + str(n) + ".png"
        # Save image
        bkg_w_icon.save(fp=output_fp, format="png")
        # Save annotation data
        annotations.append({
            "path": output_fp, 
            "annotations": ann
        })
        print(n)
        n += 1

In [ ]:
# Save annotations
with open("annotations.json", "w") as f:
    f.write(json.dumps(annotations))

In [ ]:
# Load images and annotations to sframes
images = tc.load_images("NewTrainingImages")
annots = tc.SFrame(pd.DataFrame(annotations))

In [ ]:
images.head()

In [ ]:
annots.head()

In [ ]:
# Sort them and then join
images = images.sort("path")
annots = annots.sort("path")

images = images.join(annots)

In [ ]:
images.head()

In [ ]:
# Preview the annotations
#images['image_w_annotation'] = tc.object_detector.util.draw_bounding_boxes(images['image'], images['annotations'])

#images['image_w_annotation'][0].show()

In [ ]:
# Save out sframe to use for training!
images[['image', 'path', 'annotations']].save("dashlight_images.sframe")